# Advent of Code 2016
Day 10

## Input file
First we load the input file.

In [1]:
import pandas as pd

infile = 'day10-input.txt'

df = pd.read_csv(infile, header=None, sep=';', engine='python')
df.columns = ['input']

df.head(4)

,input
0,bot 75 gives low to bot 145 and high to bot 95
1,bot 116 gives low to bot 157 and high to bot 197
2,bot 185 gives low to bot 57 and high to bot 139
3,bot 202 gives low to bot 105 and high to bot 209


In [98]:
infile = 'day10-test.txt'

dft = pd.read_csv(infile, header=None, sep=';', engine='python')
dft.columns = ['input']

dft

,input
0,value 5 goes to bot 2
1,bot 2 gives low to bot 1 and high to bot 0
2,value 3 goes to bot 1
3,bot 1 gives low to output 1 and high to bot 0
4,bot 0 gives low to output 2 and high to output 0
5,value 2 goes to bot 2


In [114]:
# Determine if an input is an initial value and parse it
def init (s):
    header = 'value '
    if s[:len(header)] == header:
        data = s[len(header):].split(' goes to bot ')
        value = int(data[0])
        bot = int(data[1])
        return (bot, value)
    else:
        return None

In [115]:
# Determine if an input is an order and parse it
def order (s):
    header = 'bot '
    if s[:len(header)] == header:
        tmp = s[len(header):].split(' gives low to ')
        bot = int(tmp[0])
        tmp2 = tmp[1].split(' and high to ')
        low = tmp2[0]
        high = tmp2[1]
        if low[:len(header)] == header:
            botl = int(low[len(header):])
            outl = None
        else:
            botl = None
            outl = int(low[7:])
        if high[:len(header)] == header:
            both = int(high[len(header):])
            outh = None
        else:
            both = None
            outh = int(high[7:])
        return (bot, (botl, outl), (both, outh))
    else:
        return None

In [116]:
# Get the initial bots hand
def initbots(df):
    bots = {}
    for index, row in df.iterrows():
        if init(row['input']) != None:
            if init(row['input'])[0] in bots:
                tmp = bots[init(row['input'])[0]]
                tmp.append(init(row['input'])[1])
                bots.update({init(row['input'])[0] : tmp})
            else:
                bots.update({init(row['input'])[0] : [init(row['input'])[1]]})
    return bots

In [117]:
# Loads bots instructions
def loadinstr(df):
    instr = {}
    for index, row in df.iterrows():
        if order(row['input']) != None:
            instr.update({order(row['input'])[0] : [order(row['input'])[1], order(row['input'])[2]]})
    return instr

In [168]:
import copy

# Run the robot factory
def runfactory(df, verbose=True):

    instr = loadinstr(df)
    bots = initbots(df)
    outputs = {}

    updated = True
    while updated:
        updated = False
        if verbose: print('***Starting a cycle***')
        if verbose: print ('Bots hand: {}'.format(bots))
        nbots = copy.deepcopy(bots)
        for b,h in bots.items():
            if len(h)==2:
                if verbose: print('  Bot {} has value-{} and value-{} chips.'.format(b,h[0],h[1]))
                if 61 in h and 17 in h: print('Part 1: Bot {} is comparing value-61 and value-17 chips.'.format(b))
                t = instr[b][0][0]
                if t != None:
                    tmp = nbots[t] if t in nbots else []
                    tmp.append(min(h))
                    nbots.update({t: tmp})
                    if verbose: print('    Value-{} chip goes to bot {}.'.format(min(h),t))
                t = instr[b][0][1]
                if t != None:
                    tmp = outputs[t] if t in outputs else []
                    tmp.append(min(h))
                    outputs.update({t: tmp})
                    if verbose: print('    Value-{} chip goes to output {}.'.format(min(h),t))
                t = instr[b][1][0]
                if t != None:
                    tmp = nbots[t] if t in nbots else []
                    tmp.append(max(h))
                    nbots.update({t: tmp})
                    if verbose: print('    Value-{} chip goes to bot {}.'.format(max(h),t))
                t = instr[b][1][1]
                if t != None:
                    tmp = outputs[t] if t in outputs else []
                    tmp.append(max(h))
                    outputs.update({t: tmp})
                    if verbose: print('    Value-{} chip goes to output {}.'.format(max(h),t))
                nbots.update({b:[]})
                updated = True
        bots = copy.deepcopy(nbots)

    if verbose: print ('Final output content: {}'.format(outputs))
    return outputs

## Test the factory

In [169]:
out = runfactory(dft)

***Starting a cycle***
Bots hand: {1: [3], 2: [5, 2]}
  Bot 2 has value-5 and value-2 chips.
    Value-2 chip goes to bot 1.
    Value-5 chip goes to bot 0.
***Starting a cycle***
Bots hand: {0: [5], 1: [3, 2], 2: []}
  Bot 1 has value-3 and value-2 chips.
    Value-2 chip goes to output 1.
    Value-3 chip goes to bot 0.
***Starting a cycle***
Bots hand: {0: [5, 3], 1: [], 2: []}
  Bot 0 has value-5 and value-3 chips.
    Value-3 chip goes to output 2.
    Value-5 chip goes to output 0.
***Starting a cycle***
Bots hand: {0: [], 1: [], 2: []}
Final output content: {0: [5], 1: [2], 2: [3]}


## Run the factory

In [170]:
out = runfactory(df, False)
print('Part 2: Multiplication of values of chips in output 1,2 and 3 is: {}'.format(out[0][0]*out[1][0]*out[2][0]))


Part 1: Bot 141 is comparing value-61 and value-17 chips.
Part 2: Multiplication of values of chips in output 1,2 and 3 is: 1209
